In [7]:
import pandas as pd
import numpy as np
import joblib

# Cargar los datos
df = pd.read_csv('../../data/processed/JNJ_clean.csv')

# Crear los conjuntos de entrenamiento y prueba, basados en la columna 'Date'
train_df = df[df['Date'] <= '2025-02-28']
test_df = df[df['Date'] >= '2025-03-01']

# Para la simulación se utiliza test_df (ya que contiene las columnas de precios necesarios)
# Además, para métricas de clasificación se pueden definir X_test e y_test (sin columnas de precios)
X_test = test_df.drop(columns=['target', 'Close', 'Open', 'High', 'Low', 'Date'])
y_test = test_df['target']

# Parámetros financieros
initial_capital = 1000          # Capital inicial en dólares
investment_fraction = 0.25      # Fracción del capital a invertir por operación
investment_per_trade = initial_capital * investment_fraction

print("Dimensiones de test_df:", test_df.shape)


Dimensiones de test_df: (30, 34)


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Lista de nombres de modelos (deben coincidir con los archivos guardados)
model_names = ["DecisionTree", "RandomForest", "AdaBoost", "GradientBoosting", "LightGBM", "XGBoost", "CatBoost"]

# Definir un rango de thresholds para evaluar (por ejemplo, de 0.40 a 0.75 en incrementos de 0.05)
thresholds = np.arange(0.40, 0.80, 0.10)

# Lista para almacenar los resultados de la simulación para cada modelo y threshold
simulation_results = []

for model_name in model_names:
    # Cargar el modelo
    model_path = f'../../models/{model_name}_JNJ.pkl'
    model = joblib.load(model_path)
    
    # Obtener las probabilidades de la clase positiva (se asume predict_proba)
    try:
        y_proba = model.predict_proba(X_test)[:, 1]
    except AttributeError:
        # Si el modelo no tiene predict_proba, se usa predict (aunque no ofrece probabilidades)
        y_proba = model.predict(X_test)
    
    for thresh in thresholds:
        # Generar la señal de trading: 1 si la probabilidad supera el threshold, 0 en caso contrario
        y_pred = (y_proba > thresh).astype(int)
        
        # Simulación financiera:
        temp_df = test_df.copy()
        temp_df["pred"] = y_pred
        # Calcular el precio de cierre del siguiente período (shift -1)
        temp_df["next_close"] = temp_df["Close"].shift(-1)
        # Porcentaje de retorno: diferencia entre el siguiente cierre y el actual
        temp_df["return_pct"] = (temp_df["next_close"] - temp_df["Close"]) / temp_df["Close"]
        
        # Seleccionar las filas donde la señal es de compra (pred == 1)
        trade_df = temp_df[temp_df["pred"] == 1].copy()
        # Eliminar filas con 'NaN' en next_close (última fila sin dato futuro)
        trade_df = trade_df.dropna(subset=["next_close"])
        # Calcular la ganancia para cada trade
        trade_df["profit"] = investment_per_trade * trade_df["return_pct"]
        
        total_profit = trade_df["profit"].sum()
        num_trades = len(trade_df)
        wins = (trade_df["profit"] > 0).sum()
        losses = (trade_df["profit"] < 0).sum()
        final_capital = initial_capital + total_profit
        
        simulation_results.append({
            "Modelo": model_name,
            "Threshold": round(thresh, 2),
            "Trades": num_trades,
            "Wins": wins,
            "Losses": losses,
            "Profit ($)": round(total_profit, 2),
            "Final Capital": round(final_capital, 2)
        })

# Convertir los resultados a un DataFrame para su análisis
df_simulation = pd.DataFrame(simulation_results)
df_simulation


,Modelo,Threshold,Trades,Wins,Losses,Profit ($),Final Capital
0,DecisionTree,0.4,23,11,12,-31.72,968.28
1,DecisionTree,0.5,10,5,5,-2.93,997.07
2,DecisionTree,0.6,10,5,5,-2.93,997.07
3,DecisionTree,0.7,0,0,0,0.00,1000.00
4,RandomForest,0.4,28,16,12,-20.26,979.74
5,RandomForest,0.5,15,9,6,0.59,1000.59
6,RandomForest,0.6,5,3,2,-0.16,999.84
7,RandomForest,0.7,0,0,0,0.00,1000.00
8,AdaBoost,0.4,22,13,9,-2.37,997.63
9,AdaBoost,0.5,12,6,6,2.88,1002.88


In [9]:
# Ordenar los resultados por 'Profit ($)' de forma descendente
df_simulation_sorted = df_simulation.sort_values(by="Profit ($)", ascending=False)

print("Comparación de la simulación financiera para todos los modelos:")
print(df_simulation_sorted.to_string(index=False))


Comparación de la simulación financiera para todos los modelos:
          Modelo  Threshold  Trades  Wins  Losses  Profit ($)  Final Capital
        LightGBM        0.5      15    10       5       11.93        1011.93
         XGBoost        0.5      13     9       4       11.88        1011.88
        LightGBM        0.7       6     4       2       10.45        1010.45
        LightGBM        0.4      17    11       6        8.60        1008.60
        LightGBM        0.6      10     6       4        6.36        1006.36
GradientBoosting        0.5      12     7       5        3.58        1003.58
GradientBoosting        0.6      11     6       5        3.53        1003.53
GradientBoosting        0.7      11     6       5        3.53        1003.53
        AdaBoost        0.5      12     6       6        2.88        1002.88
         XGBoost        0.6       6     3       3        2.87        1002.87
    RandomForest        0.5      15     9       6        0.59        1000.59
        CatB